In [2]:
import pandas as pd
import re

In [3]:
filename = 'october2020.csv'
df = pd.read_csv(filename)
df.drop(df.columns[0], axis=1, inplace=True)
df.drop_duplicates('link', inplace=True)
df.set_index('link', inplace=True)
df.head()

,date,post,title
link,,,
/r/mechmarket/comments/jlwj7h/sgh_new_and_unbuilt_tx66_white_gmk_coral_gmk/,1.604205e+09,[TX66 Timestamp](https://imgur.com/a/XvaXEQD)\...,"[SG][H] New and Unbuilt TX66 White, GMK Coral,..."
/r/mechmarket/comments/jlwi4o/usca_h_mojo60_ewhite_dz60rgbansi_v2_hotswap_pcb/,1.604205e+09,[**Timestamp**](https://imgur.com/a/LCN5jst)\n...,"[US-CA] [H] Mojo60 ewhite, DZ60RGB-ANSI v2 Hot..."
/r/mechmarket/comments/jlvr38/usca_h_dsa_granite_text_modifiers_dsa_high/,1.604201e+09,Timestamps:\n\n[https://drive.google.com/file/...,"[US-CA] [H] DSA Granite Text Modifiers, DSA Hi..."
/r/mechmarket/comments/jlvjt9/uscah_gmk_fuyu_kuro_shiro_w_paypal_cash/,1.604200e+09,Stamp: https://imgur.com/a/qAbzDg3\n\nGMK Fuyu...,"[US-CA][H] GMK Fuyu, Kuro Shiro [W] Paypal, Cash"
/r/mechmarket/comments/jlvcg1/uscahbuilt_kbd75v2_nk65ee_blumen_and_smoke_gmk/,1.604199e+09,[TIMESTAMP](https://imgur.com/a/zwALQZ3)\n\nHa...,"[US-CA][H]built KBD75V2, NK65ee blumen and smo..."


In [4]:
df['post_lower'] = df['post'].str.lower()
df['title_lower'] = df['title'].str.lower()
only_gmk = df[df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')].copy()
not_gmk = df[(~(df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')))].copy()

only_gmk.shape, df.shape

((1181, 5), (1188, 5))

In [5]:
not_gmk.head()

,date,post,title,post_lower,title_lower
link,,,,,
/r/mechmarket/comments/jjr74m/usca_h_fc660c_pebble_switches_w_paypal/,1.603903e+09,[Timestamp](https://imgur.com/a/UsZRmq8)\n\n&#...,"[US-CA] [H] FC660C, Pebble, Switches [W] PayPal",[timestamp](https://imgur.com/a/uszrmq8)\n\n&#...,"[us-ca] [h] fc660c, pebble, switches [w] paypal"
/r/mechmarket/comments/jby7ir/usca_h_rekt1800_black_kit_iris_rev3_built/,1.602802e+09,[Timestamp & Album](https://imgur.com/a/0aktko...,"[US-CA] [H] Rekt1800 Black Kit, Iris Rev3 Buil...",[timestamp & album](https://imgur.com/a/0aktko...,"[us-ca] [h] rekt1800 black kit, iris rev3 buil..."
/r/mechmarket/comments/jbjrcb/eult_h_drop_alt_zilents_v2_62g_cherry_mx_red/,1.602751e+09,[TIMESTAMP](https://imgur.com/a/p7fejyk)\n\nTo...,"[EU-LT] [H] Drop Alt, Zilents V2 (62g), Cherry...",[timestamp](https://imgur.com/a/p7fejyk)\n\nto...,"[eu-lt] [h] drop alt, zilents v2 (62g), cherry..."
/r/mechmarket/comments/j8yf6u/uspah_acrylic_tofu_gateron_inks_switch_parts/,1.602388e+09,[stamp](https://imgur.com/a/OfHba9z)\n\nBIG UP...,"[US-PA][H] Acrylic Tofu, Gateron Inks, switch ...",[stamp](https://imgur.com/a/ofhba9z)\n\nbig up...,"[us-pa][h] acrylic tofu, gateron inks, switch ..."
/r/mechmarket/comments/j6h8d0/usca_h_silver_m65a_nylon_base_artisans_bumens_etc/,1.602031e+09,removed,"[US-CA] [H] Silver M65-A (nylon base), Artisan...",removed,"[us-ca] [h] silver m65-a (nylon base), artisan..."


In [45]:
from collections import defaultdict

money_regex = r'([£€\$]\d+)|(\d+[£€\$])'
euro_to_usd = 1.2
pound_to_usd = 1.4

sets = ['base', 'nov', 'alpha', 'accent', 'bars', 'spacebar', 'light base', 'dark base', 'deskmat', 'rama', '40s', '40\'s', 'fourties', 'mods', 'extension']

sales_data = []

def match_product(df):
    for row in df.itertuples():
        s = row.post_lower.split("\n")
        #product = row.product_name.lower()
        #print(row[0][1])
        for l in s:
            low = l.lower()
            if "gmk " in low and ('~~' in low or 'sold' in low):
                a = low.split("gmk ")[1]
                product_name = "gmk " + re.split(r"[^\w\+\.]", a)[0]
                if "gmk stab" in product_name:
                    continue
                matches = re.split(money_regex, low)
                
                temp_data = {}
                if len(matches) > 1:
                    #print(f"FULL LINE: {low}")
                    hadBase = False
                    allKits = []
                    for i in range(0, len(matches)-1, 3):
                        curr_price = int(matches[i+1][1:] if matches[i+1] else matches[i+2][:-1])
                        currency = matches[i+1][0] if matches[i+1] else matches[i+2][-1]
                        if currency == "€":
                            curr_price = round(curr_price * euro_to_usd)
                        if currency == "£":
                            curr_price = round(curr_price * pound_to_usd)
                        curr_str = matches[i]
                        kits = []
                        removeBase = False
                        empty=True
                        for x in sets:
                            if x in curr_str:
                                empty=False
                                if x == 'nov':
                                    kits.append('novelties')
                                elif x == 'light base' or x == 'dark base':
                                    kits.insert(0, f'{x}')
                                    removeBase = True
                                elif x == 'bars' or x == 'spacebar':
                                    if 'spacebars' not in kits:
                                        kits.append('spacebars')
                                elif x in ['40s', '40\'s', 'fourties']:
                                    kits.append('40s')
                                else:
                                    kits.append(x)
                        products = product_name
                        if i == 0 and not kits:
                            kits = ['base']
                        for x in kits:
                            if removeBase and x == 'base':
                                continue
                            products += f', {x}'
                        if i == 0:
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        
                        if kits and i > 0:
                            #print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                            #all_prices[temp_data['products']].append(temp_data['price'])
                            sales_data.append([row[0], temp_data['products'], temp_data['price'], row.date])
                            
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        else:
                            temp_data["price"] = min(temp_data["price"], curr_price)
                    #print(type(temp_data['products']))
                    quit = False
                    if 'base' not in temp_data['products']:
                        temp = low.split('gmk')[1]
                        for x in temp.split(' '):
                            for y in sets[1:]:
                                if y in x:
                                    quit = True
                            if quit: break
                                    
                            if '+' in x or ',' in x:

                                if x == 'olivia++':
                                    break
                                # print(low)
                                # print(row[0])
                                # print(x)
                                a = temp_data['products']
                                temp_data['products'] = a[:a.index(',')] + ', base' + a[a.index(','):]

                                break
                            



                    #print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                    #all_prices[temp_data['products']].append(temp_data['price'])
                    sales_data.append([row[0], temp_data['products'], temp_data['price'], row.date])
                

for i, new_df in only_gmk.groupby(level=0):
    match_product(new_df)
    
#print(sales_data)

~~gmk metropolis base - [this](https://i.imgur.com/afpkwe0.jpg) set hasn't seen a great deal of use and, though is a interesting set, happens to get swapped out eventually. it hasn't shined as of yet and is just the base kit. asking price is **£185/$240**~~**sold**
~~gmk calm depths assembly and space bar kit - bought [this](https://i.imgur.com/afpkwe0.jpg) on the off change i use some otho/splergo board... never did. no shine and i haven't used it at all. will not ship in it's tray. asking price for both kits is **£140/$180**~~**sold**
|~~gmk fro.yo (base kit)~~|~~sealed~~|~~£200 + shipping~~ sold|
|gmk cafe base sold!!! + black (dark alphas) kit|used sparingly on my m60 for about 10 days before realising i need a 65% board. black kit is opened but unused (apart from the 7u spacebar). shipped in original gmk packaging (or bags if you want to save on shipping)|£210 and £110 respectively, or buy both for £310|
|gmk cafe base sold!!! + black (dark alphas) kit|used sparingly on my m60 for

In [44]:
sales_df = pd.DataFrame(sales_data, columns=['link', 'product', 'price', 'date'])
sales_df['date'] = pd.to_datetime(sales_df['date'], unit = 's')
sales_df.to_csv('results_2' + filename)